In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from modules._import_helper import safe_import_from

set_seed = safe_import_from('00_repo_standards.src.mlphys_core', 'set_seed')
ParticleFilter = safe_import_from(
    '04_time_series_state_space.src.particle_filter',
    'ParticleFilter'
)

set_seed(42)
plt.style.use('default')

output_dir = Path('modules/04_time_series_state_space/reports/nb05_neural_decoding')
output_dir.mkdir(parents=True, exist_ok=True)

print("✓ Setup complete")

## 3. Generate Synthetic Neural Data

Simulate latent state + Poisson spike observations.

In [ ]:
def simulate_neural_data(n_steps=200, Q=0.1, C=1.0, baseline_rate=5.0, 
                          bin_size_ms=50, seed=42):
    """
    Simulate latent state + Poisson spike counts.
    
    Args:
        n_steps: Number of time bins
        Q: Process noise variance (latent state diffusion)
        C: Encoding gain (state -> firing rate)
        baseline_rate: Mean firing rate when x=0 (Hz)
        bin_size_ms: Time bin size in milliseconds
        seed: Random seed
        
    Returns:
        x_true: True latent states (n_steps,)
        y_obs: Observed spike counts (n_steps,)
        lambda_true: True firing rates (n_steps,)
    """
    rng = np.random.default_rng(seed)
    
    # Convert baseline rate to expected counts per bin
    # baseline_rate is in Hz, bin_size_ms in ms
    b = np.log(baseline_rate * bin_size_ms / 1000)  # log of expected counts at x=0
    
    # Generate latent state (random walk)
    x_true = np.zeros(n_steps)
    x_true[0] = 0.0  # Start at baseline
    
    for t in range(1, n_steps):
        x_true[t] = x_true[t-1] + rng.normal(0, np.sqrt(Q))
    
    # Generate firing rates and spike counts
    lambda_true = np.exp(C * x_true + b)  # Always positive!
    y_obs = rng.poisson(lambda_true)  # Spike counts
    
    return x_true, y_obs, lambda_true, b

# Generate data
n_steps = 200
Q = 0.05  # Process noise
C = 0.8   # Encoding gain
baseline_rate = 10.0  # 10 Hz baseline
bin_size_ms = 50  # 50 ms bins

x_true, y_obs, lambda_true, b = simulate_neural_data(
    n_steps=n_steps, Q=Q, C=C, baseline_rate=baseline_rate, 
    bin_size_ms=bin_size_ms, seed=42
)

print(f"Generated {n_steps} time bins ({n_steps * bin_size_ms / 1000:.1f} seconds)")
print(f"Latent state range: [{x_true.min():.2f}, {x_true.max():.2f}]")
print(f"Spike count range: [{y_obs.min()}, {y_obs.max()}]")
print(f"Mean firing rate: {lambda_true.mean():.2f} spikes/bin ({lambda_true.mean() * 1000 / bin_size_ms:.1f} Hz)")

In [ ]:
# Visualize synthetic data
t = np.arange(n_steps) * bin_size_ms / 1000  # Time in seconds

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Latent state
axes[0].plot(t, x_true, 'b-', linewidth=2, label='True latent state $x_t$')
axes[0].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_ylabel('Latent State', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Neural State-Space Model: Latent State → Firing Rate → Spikes', fontsize=13)

# Firing rate
axes[1].plot(t, lambda_true, 'orange', linewidth=2, label=r'Firing rate $\lambda_t = \exp(Cx_t + b)$')
axes[1].axhline(np.exp(b), color='gray', linestyle='--', alpha=0.5, label='Baseline rate')
axes[1].set_ylabel('Firing Rate\n(spikes/bin)', fontsize=12)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

# Spike counts (as raster-like)
axes[2].bar(t, y_obs, width=bin_size_ms/1000*0.8, color='green', alpha=0.7, label='Observed spike counts')
axes[2].set_xlabel('Time (s)', fontsize=12)
axes[2].set_ylabel('Spike Count', fontsize=12)
axes[2].legend(fontsize=11)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'neural_data_simulation.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n💡 Notice: When x_t ↑ → λ_t ↑ → more spikes observed")

## 4. Poisson Observation Likelihood

For decoding, we need $P(y_t | x_t)$:

$$P(y_t | x_t) = \frac{\lambda_t^{y_t} e^{-\lambda_t}}{y_t!}, \quad \lambda_t = \exp(C x_t + b)$$

In log form (numerically stable):
$$\log P(y_t | x_t) = y_t \log(\lambda_t) - \lambda_t - \log(y_t!)$$

In [ ]:
from scipy.special import gammaln  # log(n!) = gammaln(n+1)

def poisson_log_likelihood(y, x, C, b):
    """
    Log-likelihood of Poisson observation.
    
    P(y | x) = λ^y * exp(-λ) / y!
    log P(y | x) = y * log(λ) - λ - log(y!)
    
    where λ = exp(C*x + b)
    """
    log_lambda = C * x + b
    lambda_val = np.exp(log_lambda)
    
    # log P(y|x) = y * log(λ) - λ - log(y!)
    log_lik = y * log_lambda - lambda_val - gammaln(y + 1)
    return log_lik

# Visualize likelihood for different spike counts
x_range = np.linspace(-2, 2, 200)
spike_counts = [0, 1, 3, 5]

fig, ax = plt.subplots(figsize=(10, 6))
for y in spike_counts:
    log_lik = poisson_log_likelihood(y, x_range, C, b)
    # Convert to probability (normalized for visualization)
    prob = np.exp(log_lik - log_lik.max())
    ax.plot(x_range, prob, linewidth=2, label=f'y = {y} spikes')

ax.set_xlabel('Latent State $x$', fontsize=12)
ax.set_ylabel('$P(y | x)$ (normalized)', fontsize=12)
ax.set_title('Poisson Likelihood: More Spikes → Higher Estimated State', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'poisson_likelihood.png', dpi=120, bbox_inches='tight')
plt.show()

print("💡 Observing more spikes shifts our belief about x_t to higher values.")

## 5. Particle Filter for Neural Decoding

Since Poisson is non-Gaussian, we use **particle filter**.

In [ ]:
class NeuralParticleFilter:
    """
    Particle filter for Poisson-observation state-space model.
    
    State model: x_t = x_{t-1} + w_t, w_t ~ N(0, Q)
    Observation: y_t ~ Poisson(exp(C*x_t + b))
    """
    
    def __init__(self, n_particles, Q, C, b, seed=42):
        self.n_particles = n_particles
        self.Q = Q
        self.C = C
        self.b = b
        self.rng = np.random.default_rng(seed)
        
    def run(self, observations):
        """
        Run particle filter on spike count observations.
        
        Returns:
            x_est: Estimated states (posterior mean)
            x_particles_history: All particles over time
            weights_history: Particle weights over time
            ess_history: Effective sample size over time
        """
        n_steps = len(observations)
        
        # Initialize particles
        particles = self.rng.normal(0, 1, self.n_particles)
        weights = np.ones(self.n_particles) / self.n_particles
        
        # Storage
        x_est = np.zeros(n_steps)
        x_particles_history = np.zeros((n_steps, self.n_particles))
        weights_history = np.zeros((n_steps, self.n_particles))
        ess_history = np.zeros(n_steps)
        
        for t in range(n_steps):
            # 1. Predict: propagate particles through state model
            particles = particles + self.rng.normal(0, np.sqrt(self.Q), self.n_particles)
            
            # 2. Update: weight by Poisson likelihood
            y_t = observations[t]
            log_weights = poisson_log_likelihood(y_t, particles, self.C, self.b)
            
            # Normalize weights (in log space for stability)
            log_weights = log_weights - np.max(log_weights)  # Prevent overflow
            weights = np.exp(log_weights)
            weights = weights / weights.sum()
            
            # 3. Estimate: weighted mean
            x_est[t] = np.sum(weights * particles)
            
            # Store for analysis
            x_particles_history[t] = particles
            weights_history[t] = weights
            
            # 4. Compute effective sample size
            ess = 1.0 / np.sum(weights**2)
            ess_history[t] = ess
            
            # 5. Resample if ESS too low
            if ess < self.n_particles / 2:
                indices = self.rng.choice(
                    self.n_particles, size=self.n_particles, 
                    replace=True, p=weights
                )
                particles = particles[indices]
                weights = np.ones(self.n_particles) / self.n_particles
        
        return x_est, x_particles_history, weights_history, ess_history

# Run particle filter
n_particles = 500
pf = NeuralParticleFilter(n_particles=n_particles, Q=Q, C=C, b=b, seed=42)
x_est, particles_hist, weights_hist, ess_hist = pf.run(y_obs)

print(f"Particle filter with {n_particles} particles")
print(f"Mean ESS: {ess_hist.mean():.1f} ({ess_hist.mean()/n_particles*100:.1f}% of particles)")

In [ ]:
# Visualize decoding results
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Decoded vs True
axes[0].plot(t, x_true, 'b-', linewidth=2, label='True latent state')
axes[0].plot(t, x_est, 'r--', linewidth=2, label='Decoded (PF estimate)')
axes[0].fill_between(t, x_est - 0.5, x_est + 0.5, color='red', alpha=0.2)
axes[0].set_ylabel('Latent State', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Neural Decoding: Particle Filter on Poisson Observations', fontsize=13)

# Spike observations
axes[1].bar(t, y_obs, width=bin_size_ms/1000*0.8, color='green', alpha=0.7)
axes[1].set_ylabel('Spike Count', fontsize=12)
axes[1].grid(True, alpha=0.3)

# ESS
axes[2].plot(t, ess_hist, 'purple', linewidth=2)
axes[2].axhline(n_particles/2, color='red', linestyle='--', label='Resample threshold')
axes[2].set_xlabel('Time (s)', fontsize=12)
axes[2].set_ylabel('Effective Sample Size', fontsize=12)
axes[2].legend(fontsize=11)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'decoding_results.png', dpi=120, bbox_inches='tight')
plt.show()

# Compute error
rmse = np.sqrt(np.mean((x_true - x_est)**2))
corr = np.corrcoef(x_true, x_est)[0, 1]
print(f"\n📊 Decoding Performance:")
print(f"   RMSE: {rmse:.4f}")
print(f"   Correlation: {corr:.4f}")

## 6. Binning Trade-offs

**Key question:** How does bin size affect decoding?

- **Small bins** (e.g., 10 ms): High temporal resolution, but mostly 0-1 spikes (low SNR)
- **Large bins** (e.g., 200 ms): More spikes per bin (better SNR), but lose temporal detail

In [ ]:
def run_decoding_experiment(bin_size_ms, baseline_rate=10.0, n_seconds=10, seed=42):
    """Run complete simulation + decoding for given bin size."""
    n_steps = int(n_seconds * 1000 / bin_size_ms)
    
    # Simulate
    x_true, y_obs, lambda_true, b = simulate_neural_data(
        n_steps=n_steps, Q=0.05, C=0.8, 
        baseline_rate=baseline_rate, bin_size_ms=bin_size_ms, seed=seed
    )
    
    # Decode
    pf = NeuralParticleFilter(n_particles=300, Q=0.05, C=0.8, b=b, seed=seed+1)
    x_est, _, _, _ = pf.run(y_obs)
    
    # Metrics
    rmse = np.sqrt(np.mean((x_true - x_est)**2))
    corr = np.corrcoef(x_true, x_est)[0, 1]
    mean_spikes = y_obs.mean()
    
    return {
        'bin_size_ms': bin_size_ms,
        'rmse': rmse,
        'correlation': corr,
        'mean_spikes_per_bin': mean_spikes,
        'n_bins': n_steps,
    }

# Test different bin sizes
bin_sizes = [10, 20, 50, 100, 200]
results_binning = []

print("Running binning experiment...")
for bs in bin_sizes:
    result = run_decoding_experiment(bs, seed=42)
    results_binning.append(result)
    print(f"  Bin {bs:3d} ms: RMSE={result['rmse']:.4f}, r={result['correlation']:.3f}, "
          f"mean spikes={result['mean_spikes_per_bin']:.2f}")

In [ ]:
# Visualize binning trade-off
import pandas as pd

df_bin = pd.DataFrame(results_binning)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# RMSE vs bin size
axes[0].plot(df_bin['bin_size_ms'], df_bin['rmse'], 'bo-', linewidth=2, markersize=10)
axes[0].set_xlabel('Bin Size (ms)', fontsize=12)
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('Decoding Error vs Bin Size', fontsize=13)
axes[0].grid(True, alpha=0.3)

# Correlation vs bin size
axes[1].plot(df_bin['bin_size_ms'], df_bin['correlation'], 'go-', linewidth=2, markersize=10)
axes[1].set_xlabel('Bin Size (ms)', fontsize=12)
axes[1].set_ylabel('Correlation', fontsize=12)
axes[1].set_title('Decoding Correlation vs Bin Size', fontsize=13)
axes[1].set_ylim(0, 1)
axes[1].grid(True, alpha=0.3)

# Mean spikes per bin
axes[2].bar(df_bin['bin_size_ms'], df_bin['mean_spikes_per_bin'], color='orange', alpha=0.7)
axes[2].set_xlabel('Bin Size (ms)', fontsize=12)
axes[2].set_ylabel('Mean Spikes/Bin', fontsize=12)
axes[2].set_title('Signal Strength vs Bin Size', fontsize=13)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'binning_tradeoff.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n💡 Trade-off:")
print("   Small bins → High temporal resolution but noisy (few spikes)")
print("   Large bins → More spikes but lose temporal detail")
print(f"   Optimal bin size depends on firing rate and state dynamics!")

## 7. Failure Modes

### 7.1 Low Firing Rate

In [ ]:
# Low firing rate failure
firing_rates = [1, 5, 10, 20, 50]
results_rate = []

print("Testing different firing rates...")
for rate in firing_rates:
    result = run_decoding_experiment(bin_size_ms=50, baseline_rate=rate, seed=42)
    result['baseline_rate'] = rate
    results_rate.append(result)
    print(f"  Rate {rate:2d} Hz: RMSE={result['rmse']:.4f}, r={result['correlation']:.3f}")

df_rate = pd.DataFrame(results_rate)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df_rate['baseline_rate'], df_rate['correlation'], 'ro-', linewidth=2, markersize=10)
ax.set_xlabel('Baseline Firing Rate (Hz)', fontsize=12)
ax.set_ylabel('Decoding Correlation', fontsize=12)
ax.set_title('Failure Mode: Low Firing Rate → Poor Decoding', fontsize=13)
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='Poor decoding threshold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1)

plt.tight_layout()
plt.savefig(output_dir / 'failure_low_rate.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n⚠️ Low firing rate → sparse observations → poor state estimates")

### 7.2 Model Mismatch

In [ ]:
# Model mismatch: wrong C parameter
x_true, y_obs, lambda_true, b = simulate_neural_data(
    n_steps=200, Q=0.05, C=0.8, baseline_rate=10.0, bin_size_ms=50, seed=42
)

C_values = [0.2, 0.5, 0.8, 1.2, 2.0]  # True C = 0.8
results_mismatch = []

print("Testing model mismatch (true C = 0.8)...")
for C_assumed in C_values:
    pf = NeuralParticleFilter(n_particles=300, Q=0.05, C=C_assumed, b=b, seed=42)
    x_est, _, _, _ = pf.run(y_obs)
    
    rmse = np.sqrt(np.mean((x_true - x_est)**2))
    corr = np.corrcoef(x_true, x_est)[0, 1]
    
    results_mismatch.append({'C_assumed': C_assumed, 'rmse': rmse, 'correlation': corr})
    print(f"  C = {C_assumed:.1f}: RMSE = {rmse:.4f}, r = {corr:.3f}")

df_mismatch = pd.DataFrame(results_mismatch)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df_mismatch['C_assumed'], df_mismatch['rmse'], 'bs-', linewidth=2, markersize=10)
ax.axvline(0.8, color='red', linestyle='--', linewidth=2, label='True C = 0.8')
ax.set_xlabel('Assumed Encoding Gain C', fontsize=12)
ax.set_ylabel('RMSE', fontsize=12)
ax.set_title('Failure Mode: Wrong Model Parameters', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'failure_model_mismatch.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n⚠️ Model mismatch: incorrect parameters degrade decoding")

## 8. Key Takeaways

✅ **State-space models** naturally describe neural encoding/decoding  
✅ **Poisson observations** are appropriate for spike counts (discrete, non-negative)  
✅ **Particle filter** handles non-Gaussian likelihoods  
✅ **Binning trade-off**: temporal resolution vs. signal strength  
✅ **Failure modes**: low firing rate, model mismatch, too few particles  

**Real-world applications:**
- Brain-machine interfaces (decode intended movement)
- Sensory neuroscience (decode stimulus from neural response)
- Clinical: epilepsy prediction, Parkinson's tremor detection

---

## 9. Exercises

### Exercise 1: Vary Particle Count

**Task:** Run decoding with N = 50, 100, 200, 500, 1000 particles.  
Plot RMSE and computation time vs N. What's the practical minimum?

In [ ]:
# Your code here

### Exercise 2: Multi-Neuron Decoding

**Task:** Simulate 3 neurons with different encoding gains C = [0.5, 0.8, 1.2].  
Combine their spike counts for decoding. Does more neurons help?

In [ ]:
# Your code here

### Exercise 3: Oscillatory Latent State

**Task:** Instead of random walk, use oscillatory latent state:  
$x_t = A \sin(2\pi f t / T) + \text{noise}$  
Does the particle filter track oscillations? How does frequency affect decoding?

In [ ]:
# Your code here

---

## 10. Solutions

### Solution 1: Particle Count Study

In [ ]:
import time

x_true, y_obs, _, b = simulate_neural_data(n_steps=200, seed=42)

n_particles_list = [50, 100, 200, 500, 1000]
results_particles = []

for N in n_particles_list:
    start = time.time()
    pf = NeuralParticleFilter(n_particles=N, Q=0.05, C=0.8, b=b, seed=42)
    x_est, _, _, _ = pf.run(y_obs)
    elapsed = time.time() - start
    
    rmse = np.sqrt(np.mean((x_true - x_est)**2))
    results_particles.append({'N': N, 'rmse': rmse, 'time_ms': elapsed*1000})

df_particles = pd.DataFrame(results_particles)
print(df_particles.to_string(index=False))

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].plot(df_particles['N'], df_particles['rmse'], 'bo-', markersize=10)
axes[0].set_xlabel('Number of Particles', fontsize=12)
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('Accuracy vs Particle Count', fontsize=13)
axes[0].grid(True, alpha=0.3)

axes[1].plot(df_particles['N'], df_particles['time_ms'], 'ro-', markersize=10)
axes[1].set_xlabel('Number of Particles', fontsize=12)
axes[1].set_ylabel('Time (ms)', fontsize=12)
axes[1].set_title('Computation Time vs Particle Count', fontsize=13)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'ex1_particle_count.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n✓ ~200-500 particles often sufficient for this problem")

### Solution 2: Multi-Neuron Decoding

In [ ]:
def simulate_multi_neuron(n_steps, n_neurons, Q, C_list, baseline_rate, bin_size_ms, seed=42):
    """Simulate multiple neurons sharing same latent state."""
    rng = np.random.default_rng(seed)
    
    # Shared latent state
    x_true = np.zeros(n_steps)
    for t in range(1, n_steps):
        x_true[t] = x_true[t-1] + rng.normal(0, np.sqrt(Q))
    
    b = np.log(baseline_rate * bin_size_ms / 1000)
    
    # Each neuron has different encoding gain
    y_all = np.zeros((n_neurons, n_steps))
    for i, C in enumerate(C_list):
        lambda_i = np.exp(C * x_true + b)
        y_all[i] = rng.poisson(lambda_i)
    
    return x_true, y_all, b

class MultiNeuronPF:
    """Particle filter for multiple Poisson neurons."""
    def __init__(self, n_particles, Q, C_list, b, seed=42):
        self.n_particles = n_particles
        self.Q = Q
        self.C_list = C_list
        self.b = b
        self.rng = np.random.default_rng(seed)
    
    def run(self, y_all):
        n_neurons, n_steps = y_all.shape
        particles = self.rng.normal(0, 1, self.n_particles)
        x_est = np.zeros(n_steps)
        
        for t in range(n_steps):
            particles = particles + self.rng.normal(0, np.sqrt(self.Q), self.n_particles)
            
            # Combine likelihoods from all neurons
            log_weights = np.zeros(self.n_particles)
            for i, C in enumerate(self.C_list):
                log_weights += poisson_log_likelihood(y_all[i, t], particles, C, self.b)
            
            log_weights -= np.max(log_weights)
            weights = np.exp(log_weights)
            weights /= weights.sum()
            
            x_est[t] = np.sum(weights * particles)
            
            if 1.0 / np.sum(weights**2) < self.n_particles / 2:
                indices = self.rng.choice(self.n_particles, self.n_particles, replace=True, p=weights)
                particles = particles[indices]
        
        return x_est

# Compare 1 vs 3 neurons
C_list = [0.5, 0.8, 1.2]
x_true, y_all, b = simulate_multi_neuron(200, 3, Q=0.05, C_list=C_list, 
                                          baseline_rate=10, bin_size_ms=50, seed=42)

# Single neuron
pf1 = NeuralParticleFilter(300, 0.05, C_list[1], b, seed=42)
x_est_1 = pf1.run(y_all[1])[0]
rmse_1 = np.sqrt(np.mean((x_true - x_est_1)**2))

# Three neurons
pf3 = MultiNeuronPF(300, 0.05, C_list, b, seed=42)
x_est_3 = pf3.run(y_all)
rmse_3 = np.sqrt(np.mean((x_true - x_est_3)**2))

print(f"Single neuron RMSE: {rmse_1:.4f}")
print(f"Three neurons RMSE: {rmse_3:.4f}")
print(f"\n✓ More neurons improve decoding by {(rmse_1-rmse_3)/rmse_1*100:.1f}%")

### Solution 3: Oscillatory State

In [ ]:
def simulate_oscillatory(n_steps, freq_hz, amplitude, Q_noise, C, baseline_rate, bin_size_ms, seed=42):
    """Oscillatory latent state + Poisson observations."""
    rng = np.random.default_rng(seed)
    
    dt = bin_size_ms / 1000  # seconds per bin
    t = np.arange(n_steps) * dt
    
    # Oscillatory state with noise
    x_true = amplitude * np.sin(2 * np.pi * freq_hz * t) + rng.normal(0, np.sqrt(Q_noise), n_steps)
    
    b = np.log(baseline_rate * bin_size_ms / 1000)
    lambda_t = np.exp(C * x_true + b)
    y_obs = rng.poisson(lambda_t)
    
    return t, x_true, y_obs, b

# Test different frequencies
frequencies = [0.5, 1, 2, 5]  # Hz
fig, axes = plt.subplots(len(frequencies), 1, figsize=(14, 3*len(frequencies)), sharex=True)

for i, freq in enumerate(frequencies):
    t_osc, x_true_osc, y_obs_osc, b_osc = simulate_oscillatory(
        n_steps=200, freq_hz=freq, amplitude=1.0, Q_noise=0.01, 
        C=0.8, baseline_rate=10, bin_size_ms=50, seed=42
    )
    
    pf = NeuralParticleFilter(300, Q=0.1, C=0.8, b=b_osc, seed=42)
    x_est_osc, _, _, _ = pf.run(y_obs_osc)
    
    corr = np.corrcoef(x_true_osc, x_est_osc)[0, 1]
    
    axes[i].plot(t_osc, x_true_osc, 'b-', linewidth=2, label='True')
    axes[i].plot(t_osc, x_est_osc, 'r--', linewidth=2, label='Decoded')
    axes[i].set_ylabel(f'{freq} Hz\nr={corr:.2f}', fontsize=11)
    axes[i].legend(loc='upper right', fontsize=9)
    axes[i].grid(True, alpha=0.3)

axes[0].set_title('Oscillatory Latent State Decoding', fontsize=13)
axes[-1].set_xlabel('Time (s)', fontsize=12)

plt.tight_layout()
plt.savefig(output_dir / 'ex3_oscillatory.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n✓ PF tracks oscillations, but high frequencies require smaller bins!")

---

## Summary Report

In [ ]:
summary = f"""
# Notebook 05: Neural Time Series Decoding - Summary

**Date:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

## Key Results

1. **State-Space Neural Model:**
   - Latent: random walk (x_t = x_{{t-1}} + noise)
   - Observation: Poisson(exp(C·x + b))

2. **Decoding Performance:**
   - RMSE: {rmse:.4f}
   - Correlation: {corr:.4f}

3. **Binning Trade-off:**
   - Small bins: high resolution, low SNR
   - Large bins: high SNR, low resolution

4. **Failure Modes:**
   - Low firing rate → sparse observations
   - Model mismatch → biased estimates

## Outputs
- neural_data_simulation.png
- poisson_likelihood.png
- decoding_results.png
- binning_tradeoff.png
- failure_*.png

## Applications
- Brain-machine interfaces
- Sensory neuroscience
- Clinical neural monitoring
"""

with open(output_dir / 'summary.md', 'w') as f:
    f.write(summary)

print("\n" + "="*60)
print("✓ Notebook 05 Complete!")
print("="*60)
print(f"Outputs saved to: {output_dir}")